## Manual data 358

In [11]:
import pandas as pd
data = pd.read_csv('data/label_ID6.csv')

In [12]:
def coalesce_rating(df):
    df['rating'] = None
    for c in data.columns:
        if c.startswith('rating'):
            df['rating'] = df['rating'].fillna(df[c])
    df['rating'] = df['rating'].map(lambda s: int(s[0]) if isinstance(s, str) else s)
coalesce_rating(data)
data = data[~data.rating.isna()].reset_index(drop=True)

In [13]:
data.rating.value_counts(), len(data)

(4.0    148
 5.0     71
 3.0     67
 2.0     50
 1.0     22
 Name: rating, dtype: int64,
 358)

In [17]:
lst = []
for x in data.itertuples():
    start = int(pd.Timedelta(x.start_position_scene).total_seconds() - 1)
    end = int(pd.Timedelta(x.end_position_scene).total_seconds() - 1)
    saliency = pd.read_csv(f'tmp/{x.content_id}_video/saliency_v3.csv')['logit']
    lst.append(saliency[start:end].mean())
data['univtg'] = lst
data['ytrue'] = (data.rating > 3).astype(int)
data['yhat'] = (data['univtg'] > 0.3).astype(int)
s = data.groupby(['yhat', 'ytrue']).size().rename('num').reset_index()
display(s.pivot_table(index='yhat', columns='ytrue', values='num') \
      .sort_index(ascending=False, axis=0).sort_index(ascending=False, axis=1))
print('precision', s[(s.yhat == 1)&(s.ytrue==1)]['num'].sum()/s[(s.yhat == 1)]['num'].sum())
print('recall', s[(s.yhat == 1)&(s.ytrue==1)]['num'].sum()/s[(s.ytrue == 1)]['num'].sum())

ytrue,1,0
yhat,,
1,158,105
0,61,34


precision 0.6007604562737643
recall 0.7214611872146118


In [18]:
from sklearn.metrics import average_precision_score, roc_auc_score
print('AP', average_precision_score(data['ytrue'], data['univtg']))
print('AUC', roc_auc_score(data['ytrue'], data['univtg']))

AP 0.6677192412240494
AUC 0.5513287999737196


## Like 2022

In [3]:
import pandas as pd
df = pd.read_csv('data/data2022/like2.csv')

In [4]:
from pathlib import Path
lst = []
for x in df.itertuples():
    content_id = Path(x.path).stem
    saliency = pd.read_csv(f'tmp/data2022/{content_id}/saliency_v2.csv')['logit']
    lst.append(saliency.mean())
df['univtg'] = lst

In [6]:
df['ytrue'] = (df['like'] > 7).map(int)

In [8]:
from sklearn.metrics import average_precision_score, roc_auc_score
print('AP', average_precision_score(df['ytrue'], df['univtg']))
print('AUC', roc_auc_score(df['ytrue'], df['univtg']))

AP 0.4632070051961239
AUC 0.5268992631718794
